# 텍스트 완료 요청을 스트리밍하는 방법

기본적으로 완료를 요청하면 전체 완료가 생성된 후 단일 응답으로 다시 전송됩니다.

긴 완료를 생성하는 경우 응답을 기다리는 데 몇 초가 걸릴 수 있습니다.

응답을 더 빨리 받으려면 완료가 생성되는 동안 '스트리밍'할 수 있습니다. 이렇게 하면 전체 완료가 끝나기 전에 완료의 시작 부분을 인쇄하거나 처리할 수 있습니다.

완료를 스트리밍하려면 채팅 완료 또는 완료 엔드포인트를 호출할 때 `stream=True`를 설정합니다. 그러면 [데이터 전용 서버 전송 이벤트](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)로 응답을 다시 스트리밍하는 개체가 반환됩니다. `message` 필드 대신 `delta` 필드에서 청크를 추출합니다.

## 단점

프로덕션 애플리케이션에서 `stream=True`를 사용하면 부분 완료를 평가하기가 더 어려울 수 있으므로 완료 내용의 조정이 더 어려워집니다.

## 예제 코드

아래에서 이 노트북은 다음을 보여줍니다.
1. 스트리밍 텍스트 완료 응답의 모양

In [1]:
# 가져오기
import easyllm  # API 호출용
import time  # API 호출 시간 측정용

### 2. 텍스트 완료를 스트리밍하는 방법

스트리밍 API 호출을 사용하면 응답이 [이벤트 스트림](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)을 통해 증분적으로 청크로 전송됩니다. Python에서는 `for` 루프를 사용하여 이러한 이벤트를 반복할 수 있습니다.

어떻게 보이는지 살펴보겠습니다.

In [1]:
from easyllm.clients import huggingface

huggingface.prompt_builder = "llama2"

# ChatCompletion 요청
response = huggingface.Completion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    prompt="50까지 세되 각 숫자 사이에 쉼표를 넣고 줄 바꿈은 하지 마세요. 예: 1, 2, 3, ...",
    stream=True  # 이번에는 stream=True로 설정합니다.
)

for chunk in response:
    print(chunk)

{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'text': ' ', 'logprobs': 0.0}]}
{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'text': ' 물론', 'logprobs': 0.0}]}
{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'text': '!', 'logprobs': 0.0}]}
{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'text': ' 여기', 'logprobs': 0.0}]}
{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-hf', 'choices': [{'index': 0, 'text': ' 있', 'logprobs': 0.0}]}
{'id': 'hf-PokypeK2an', 'object': 'text.completion', 'created': 1691129904, 'model': 'meta-llama/Llama-2-70b-chat-h

위에서 볼 수 있듯이 스트리밍 응답에는 생성된 토큰을 보유하는 `text` 필드가 있습니다.

아래는 생성된 텍스트가 들어오는 대로 인쇄하고 `</s>` 토큰을 보면 중지하는 예제입니다.

In [2]:
from easyllm.clients import huggingface

huggingface.prompt_builder = "llama2"

# ChatCompletion 요청
response = huggingface.Completion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    prompt="10까지 세되 각 숫자 사이에 쉼표를 넣고 줄 바꿈은 하지 마세요. 예: 1, 2, 3, ...",
    stream=True  # 이번에는 stream=True로 설정합니다.
)

for chunk in response:
    print(chunk["choices"][0]["text"],end="")

  물론입니다! 여기 있습니다: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.